In [21]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [4]:
data = pd.read_excel('Users/roche_shao/Desktop/Copy of classified_data.xlsx')
data.head()

,Building Name,POSTAL CODE,latitude,longitude,Building Size,Employee(people),Transportation Data in 2023 (km),km*people,Scope 3 GHG Total Emission(t CO2e)_2023,Scope 3 GHG Total Emission(kg CO2e)_2023,gross floor Area (sqm),region
0,GUOCO TOWER,78884,1.2767,103.8452,Large,12000.0,9.187972e+07,1.102557e+12,3945.500000,3.945500e+06,129726.0,DOWNTOWN CORE
1,SINGAPORE AVIATION ACADEMY,499867,1.3883,103.9903,Large,1705.0,1.171629e+07,1.997627e+10,1032.564396,1.032564e+06,22368.0,CHANGI
2,AMNIOS,138622,1.3012,103.7930,NaN,787.0,3.721624e+06,2.928918e+09,496.525067,4.965251e+05,9518.0,QUEENSTOWN
3,CUSTOMS OPERATIONS COMMAND,648171,1.3545,103.7005,Large,1828.0,1.475434e+07,2.697094e+10,843.664957,8.436650e+05,21595.0,JURONG WEST
4,NATIONAL ARCHIVES SINGAPORE,179868,1.2930,103.8483,Medium,512.0,2.709015e+06,1.387016e+09,399.372729,3.993727e+05,8214.0,MUSEUM


In [5]:
# Checking for missing values in relevant columns
missing_data = data[['km*people', 'Scope 3 GHG Total Emission(kg CO2e)_2023', 'region']].isnull().sum()

# Summary of data by region
region_summary = data.groupby('region').agg({
    'km*people': 'mean',
    'Scope 3 GHG Total Emission(kg CO2e)_2023': 'mean',
    'Building Name': 'count'
}).rename(columns={'Building Name': 'Number of Buildings'}).reset_index()

missing_data, region_summary

(km*people                                   0
 Scope 3 GHG Total Emission(kg CO2e)_2023    0
 region                                      0
 dtype: int64,
                      region     km*people  \
 0                ANG MO KIO  1.234183e+09   
 1                     BEDOK  1.714924e+11   
 2                    BISHAN  3.392974e+10   
 3               BUKIT BATOK  3.398326e+09   
 4               BUKIT MERAH  3.985258e+10   
 5             BUKIT PANJANG  7.904576e+09   
 6               BUKIT TIMAH  1.000227e+11   
 7                    CHANGI  1.195774e+10   
 8             CHOA CHU KANG  4.542900e+11   
 9                  CLEMENTI  5.191403e+10   
 10            DOWNTOWN CORE  1.266805e+11   
 11                  GEYLANG  1.301251e+11   
 12                  HOUGANG  4.211143e+09   
 13              JURONG EAST  5.153439e+10   
 14              JURONG WEST  3.031234e+10   
 15                  KALLANG  1.672486e+10   
 16            MARINE PARADE  6.833375e+09   
 17             

In [23]:
# Excluding problematic regions and re-fitting the models for the remaining regions
excluded_regions = ['BUKIT TIMAH', 'BUKIT BATOK', 'WOODLANDS', 'ANG MO KIO', 'CHOA CHU KANG',
                    'JURONG WEST', 'MARINE PARADE', 'BEDOK', 'JURONG EAST', 'DOWNTOWN CORE'
                    ]
final_results = []

for region in data['region'].unique():
    if region not in excluded_regions:
        subset = data[data['region'] == region]
        if len(subset) > 1:  # Ensure there are enough data points for regression
            X = subset[['km*people']]
            X = sm.add_constant(X)  # Adding a constant for intercept
            y = subset['Scope 3 GHG Total Emission(kg CO2e)_2023']

            try:
                model = sm.GLM(y, X, family=sm.families.Gaussian())
                result = model.fit()
                final_results.append({
                    'Region': region,
                    'Intercept': result.params['const'],
                    'Slope': result.params['km*people'],
                    'p-value': result.pvalues['km*people'],
                    'Deviance': result.deviance
                })
            except PerfectSeparationError:
                # Catching just in case any other region has issues
                continue

# Convert results to DataFrame for better display
final_results_df = pd.DataFrame(final_results)
final_results_df.sort_values(by='Deviance', ascending=True)  # Sort by Deviance for lower is better

,Region,Intercept,Slope,p-value,Deviance
5,HOUGANG,2.112057e+05,7.060325e-05,2.001845e-22,7.765407e+09
12,BISHAN,2.561267e+05,4.028632e-06,2.003458e-04,2.869156e+10
0,CHANGI,6.848866e+05,1.832433e-05,3.335661e-01,3.514213e+10
6,TOA PAYOH,2.350990e+05,2.513924e-05,2.973383e-16,1.568568e+11
17,NEWTON,7.499311e+05,1.577698e-05,2.168602e-08,1.982869e+11
14,KALLANG,5.327176e+05,1.568579e-05,3.395918e-06,3.925715e+11
4,WESTERN WATER CATCHMENT,6.122254e+05,2.833100e-05,1.093417e-02,4.226451e+11
7,ORCHARD,4.168589e+05,3.002833e-05,1.196763e-08,1.091749e+12
15,SINGAPORE RIVER,2.509038e+05,3.744947e-05,3.858476e-05,1.197079e+12
8,ROCHOR,1.037813e+06,1.067843e-05,7.515631e-01,2.040428e+13


In [25]:
excluded_data = data[data['region'].isin(excluded_regions)]
ratios_by_region = excluded_data.groupby('region').apply(
    lambda x: x['Scope 3 GHG Total Emission(kg CO2e)_2023'].mean() / x['km*people'].mean()
)

ratios_by_region

/var/folders/hb/t4jwjlbj7_d20x36kwrpzwpc0000gn/T/ipykernel_70445/813271736.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ratios_by_region = excluded_data.groupby('region').apply(


region
ANG MO KIO       0.000235
BEDOK            0.000018
BUKIT BATOK      0.000100
BUKIT TIMAH      0.000004
CHOA CHU KANG    0.000003
DOWNTOWN CORE    0.010250
JURONG EAST      0.000081
JURONG WEST      0.000021
MARINE PARADE    0.000530
WOODLANDS        0.000049
dtype: float64

In [27]:
ratios_df = ratios_by_region.reset_index()
ratios_df.columns = ['Region', 'GHG to km*people Ratio']

# Save to Excel
output_path_ratios = 'Users/roche_shao/Desktop/Ratios_by_Region.xlsx'
ratios_df.to_excel(output_path_ratios, index=False)

output_path_ratios

'Users/roche_shao/Desktop/Ratios_by_Region.xlsx'

other model

In [22]:
# Preparing the data for training, excluding the problematic regions
filtered_data = data[~data['region'].isin(excluded_regions)]

# Using 'km*people' as the independent variable and 'Scope 3 GHG Total Emission(t CO2e)_2023' as the dependent variable
X = filtered_data[['km*people']]
y = filtered_data['Scope 3 GHG Total Emission(kg CO2e)_2023']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initializing and training the Random Forest Regressor
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

# Predicting on the test set
y_pred = random_forest.predict(X_test)

# Calculating metrics
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

y_pred

array([4.56368630e+06, 4.56099793e+05, 2.96957690e+09, 1.39044809e+06,
       7.34115470e+06, 6.75577723e+05, 3.61424006e+05, 1.70698238e+06,
       2.99897758e+10, 8.80617815e+08, 7.97822030e+05, 2.47771950e+07,
       6.89131487e+05, 3.00447575e+09, 3.23004133e+05, 6.12860295e+05,
       2.83184900e+08, 1.35629246e+06, 4.41958136e+06, 8.15008976e+05,
       4.82199081e+05, 3.30501453e+06, 8.31801621e+05, 1.12799202e+09,
       7.15803487e+06, 3.53932948e+06, 8.46257028e+05, 8.31801621e+05,
       5.32111943e+08, 7.22202858e+05, 4.56099793e+05, 2.17642875e+10,
       9.11336754e+05, 1.08918480e+06, 2.70726490e+05, 2.34401970e+08,
       1.54345906e+07, 4.34621637e+06, 9.15714648e+05, 3.79415189e+06,
       1.58403785e+06, 5.92068786e+05, 5.32111943e+08, 4.11884342e+05,
       5.39898393e+06, 6.56674655e+05, 6.34776720e+05, 2.99897758e+10,
       7.34003566e+06, 5.25666837e+06, 4.78648337e+05, 4.41958136e+06,
       2.99897758e+10, 1.62964826e+07, 9.37090864e+05, 1.07566287e+06,
      